In [26]:
from bs4 import BeautifulSoup, NavigableString
import requests
import pandas as pd

def get_soup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

In [37]:
# Extracts the links to every review edition
def get_links():
    links = list()
    h2s = soup.findAll('h2', class_='teaser_title')
    for h2 in h2s:
        links.append(h2.a['href'])    
    return links

# Gets into each review edition and into every single article to draw their data out
def extract_items(links):
   
    for link in links:
        soup = get_soup(link)

        ## html structure of 'quaderns' pages
        if("QuadernsICA" not in link):
            # get review title
            print()
            review_title = soup.find('div', class_='page page_issue').h1
            print(review_title.text.strip())
            
            # Gets review description
            review_description = soup.find('div', class_='description').p
            
            # Gets url articles & pages
            ref = soup.findAll('div', class_='title')
            pages = soup.findAll('div', class_='pages')
            for a, pag in zip(ref, pages):
                
                # Appends review_title & review_description to their respective lists for each article
                review_title_list.append(review_title.text.strip())
                review_description_list.append(review_description.text.strip())
                
                # Appends article pages to article_pages_list
                article_pages_list.append(pag.text.strip())
                
                # Explores page article
                inner_soup = get_soup(a.a['href'])
                
                # Gets article title
                article_title = inner_soup.find('h1', class_='page_title')
                article_title_list.append(article_title.text.strip())
                
                # Get article authors
                authors = list()
                article_authors = inner_soup.findAll('span', class_='name')
                for author in article_authors:
                    authors.append(author.text.strip())
                joined_string = ",".join(authors)
                article_authors_list.append(joined_string)
                
                # Get article keywords
                article_keywords = inner_soup.find('span', class_='value')
                if(article_keywords):
                    joined_string = " ".join(article_keywords.text.split())
                    article_keywords_list.append(joined_string)
                else:
                    article_keywords_list.append(None)
                
                # Get article abstract
                article_abstract = inner_soup.find('div', class_='item abstract')
                article_abstract = article_abstract.text.replace('Resum', '').strip()
                article_abstract_list.append(article_abstract)
                
                # Get article pdf
                article_pdf=inner_soup.find('a', class_='obj_galley_link pdf')['href']
                article_pdf_list.append(article_pdf)
        
                print("-" + article_title.text.strip())
        
        ## html structure of 'QuadernsICA' pages
        elif ("QuadernsICA" in link):
          
            # Gets review title
            print()
            review_title = soup.find('li', class_='active')
            print(review_title.text.strip())
            
            # Gets review description
            review_description = soup.find('div', class_='description')   
            
            # Gets url articles & pages
            ref = soup.findAll('h3', class_='media-heading')
            pages = soup.findAll('p', class_='pages') # ok
            for a, pag in zip(ref, pages):
                
                # Appends review_title to review_title_list for each article
                if(review_title):
                    review_title_list.append(review_title.text.strip())
                else:
                    review_title_list.append(None)
                
                # Appends review_description to review_description_list for each article
                if(review_description):
                    review_description_list.append(review_description.text.strip())
                else:
                    review_description_list.append(None) 
                
                # Appends article pages to article_pages_list
                article_pages_list.append(pag.text.strip())
                
                # Explores page article
                inner_soup = get_soup(a.a['href'])
            
                # Gets article title
                article_title = inner_soup.find('h1', class_='page-header')
                if(article_title):              
                    article_title_list.append(article_title.text.strip())
                else:
                    article_title_list.append(None)
                    
                # Gets article authors
                authors = list()
                article_authors = inner_soup.findAll('div', class_='author')
                if(article_authors):
                    for author in article_authors:
                        authors.append(author.find('strong').text)
                    joined_string = ",".join(authors)
                    article_authors_list.append(joined_string)
                else:
                    article_authors_list.append(None)                    
                    
                # Gets article abstract & keywords
                article_abstract = inner_soup.find('div', class_='article-abstract')
                if(article_abstract):
                    article_abstract_list.append(article_abstract.text.strip())
                    article_keywords_list.append(article_abstract.text.strip().partition("Keywords: ")[2])
                else:
                    article_abstract_list.append(None)
                    article_keywords_list.append(None)
                
                # Gets article pdf
                article_pdf=inner_soup.find('a', class_='galley-link btn btn-primary pdf')
                if(article_pdf):
                    article_pdf_list.append(article_pdf['href'])
                else:
                    article_pdf_list.append(None)
                
                print("-" + article_title.text.strip())

# initialization of fields lists
review_title_list = list()
review_description_list = list()
article_title_list = list()
article_pages_list = list()
article_authors_list = list()
article_keywords_list = list()
article_abstract_list = list()
article_pdf_list = list()

print("Start")

# first page
url_page="https://www.antropologia.cat/publicacions-ica/quaderns/"
soup = get_soup(url_page)
links = get_links()
extract_items(links)
next_page = soup.find('a', class_='next page-numbers')

# following pages loop
while(next_page):
    url_page = next_page['href']
    soup = get_soup(url_page)
    links = get_links()
    extract_items(links)
    next_page = soup.find('a', class_='next page-numbers') # ok

print("Extraction done")
print("Setting dataset\n")
print(str(len(article_title_list)) + " articles from " + str(len(set(review_title_list))) + " were retrieved")

# Creates a dataset creation and populates it with field lists data
df = pd.DataFrame({'Review title':review_title_list,
                   'Review description':review_description_list,
                   'Article title':article_title_list,
                   'Article pages':article_pages_list,
                   'Article authors':article_authors_list,
                   'Article keywords':article_keywords_list,
                   'Article abstract':article_abstract_list,
                   'Article pdf':article_pdf_list})

# Writes the files
df.to_csv('Quaderns_ICA.csv', sep='|')
df.to_excel('Quaderns_ICA.xlsx')

print("Dataset written into 'Quaderns_ICA.csv' file")
print("Dataset written into 'Quaderns_ICA.xlsx' file")
print("\nEnd")

Start

No 36 (2) (2020): Patrimonialització de l'alimentació i la producció agraria
-Patrimonialización de la alimentación y la producción agraria: una introducción
-Matonaires: de la terra a la roca. Dones pageses als pobles de Montserrat
-Patrimonialización, políticas públicas y turismo: las cocinas tradicionales en México, gestión y puesta en valor
-Sabores en valor: El caso de Baztango Zaporeak
-Producción orgánica y turismo comunitario: La experiencia de ASOPROLA (Altamira de Biolley, Costa Rica)
-Identidade quilombola e a ativação patrimonial no Povoado do Moinho
-Enogastronomía en la región vitivinícola de Querétaro, México: una perspectiva crítica sobre su patrimonialización
-Muntanyes de formatge. Transformacions productives i patrimonialització a l’Urgellet i el Baridà
-Hacia una reflexión decolonial de la alimentación en el Occidente de México
-Artificis culturals. Tradició, patrimoni, turisme

No 36 (1) (2020): Salut i espiritualitat
-Introducción: Salud y espiritualidad
-D

-Corry, Stephen (2014) Pueblos indígenas para el mundo del mañana. Círculo Rojo editorial, Madrid, 453 p. ISBN 978-84-9076-575-3
-Martínez Mauri, Mònica (directora), George Angher, Daniel Buitrago-Rosas, Euclides Campos, Igua Jiménez y Grupo Wagibler (2014) Una guía ornitológica y cultural a las aves de Gunayala / An ornithological and cultural guide to the birds of Gunayala.
-Victoria Reyes-García y Tomás Huanca (eds.) (2015) Cambio global, cambio local. La sociedad tsimane’ ante la globalización. Icaria Editorial / Institut Català d’Antropologia, Barcelona. 380 p. ISBN 978-84-9888-624-55
-En homenatge a Terence Turner

No 30 (2014): Campos imprevistos: Etnografías de Asia Oriental en el contexto académico español
-Introducción. Campos imprevistos: etnografías de Asia oriental en el contexto académico español
-Japón y la antropología de ida y vuelta
-Ir a investigar y ser investigado. El trabajo de campo en China post-crisis
-De las cocinas a las aulas. Reflexión sobre un trabajo de c

-Familias rotas y problemas de salud. La medicalización de las familias migrantes ecuatorianas
-Le nuove immigrazioni el problema di strategia dei servizi sanitari europei: un quadro introductivo

Núm. 21 (2005): Reprenent l'objecte i les seves mediacions: antropologia, art i artefacte
-Presentació
-¿Son los antropólogos ciegos frente al arte?
-Entre lo estético y lo extra-estético: paradojas de la emergencia internacional del arte contemporáneo de África
-Recuerdos de metal: Cultura material y conflicto en el siglo XX
-Com fer visible l'interior del cos o diferents maneres d'imaginar el fetus
-Making Do: Agency and objective chance in the psychogenetic portraits of Jaume Xifra
-Objectes distribuïts i consciència discontinua: l'obra d'art com a mediació transtemporal i transcultural
-The visual Arts as an Expression of Change among the Baules of Côte d'Ivoire
-Nubes y claros: Breves apuntes sobre las producciones de la institución arte dentro de las "sociedades del conocimiento"
-Obra 

-Giomar Rovira. Mujeres de maíz. La voz de las indígenas de Chiapas y la rebelión zapatista, Virus, Barcelona, 1996

Núm. 10 (1997): Núm.: 10
-Bonded Labor, coercion and capitalist development in Perú
-Vegetalismo y sistema de representaciones en el curanderismo Inga-Camentsá (pretexto para una discusión sobre los cosmovisiones prehispánicas en la sociedad contemporánea)
-La fiesta de la candelaria: tradición y modernidad en Atacama (Chile)
-Dinámica étnica, modernización y ritual en el medio rural de México
-Representación y cine etnográfico
-Enric Porqueres i Gené. Lourde Aliance. Mariage et identité chez les decendats des juifs convertits à Majorque (1435-1750), Editions Kimé, Paris. 1995
-José Luis García García. Prácticas paternalistas. Un estudio antropológico sobre los mineros asturianos. Barcelona, Ariel Antropología, 1996, 253 pp
-Ana Pelegrín. La flor de la maravilla. Juegos, recreos, retahílas, Fundación Germán Sánchez Ruipérez, Madrid, 1996
-Marta Allué. Perder la piel, Edi

-Els origens de la Institució d'Hereu a Catalunya: vers una interpretació contextual
-Breu esbós de la vida i obra de Josep Mª Batista i Roca amb motiu de la seva mort
-La Catalunya rural sota el règim de Franco: El fet de la cultura regional des de la guerra civil espanyola
-L'individualisme com a procés històric: Les investigacions d'Alan Macfarlane
-Jean Bisson: Le terre et l'homme aux Iles Baléares
-Sebastià Casanovas i Canut. Memòries d'un pagès del segle XVIII
-Lluís Casassas i Simó. Fires i Mercats a Catalunya
-Mary Douglas. Símbolos naturales. Exploraciones en cosmología
-W. A. Douglass i J. B. Aceves: Los aspectos cambiantes de la España Rural
-Max G. Gluckman: Política, derecho y ritual en la sociedad tribal
-Jaume Russinyol: le problème national catalan
-Paul Thompson: The Voice of the Past
-K. Polanyi; C. M. Arensberg i Harry W. Pearson: Comercio y Mercado en los Imperios Antiguos
Extraction done
Setting dataset

391 articles from 32 were retrieved
Dataset written into 'Qua

In [ ]:
## Recerques